In [44]:
using LinearAlgebra, Random

In [50]:
abstract type AbstractSurrogateModel end

mutable struct SGP <: AbstractSurrogateModel
    name::String
    options::Dict{String, Any}
    Z::Array{Float64, 2}
    woodbury_data::Dict{String, Any}
    optimal_par::Dict{String, Any}
    optimal_noise::Float64
end

ErrorException: invalid redefinition of constant Main.SGP

In [51]:
function SGP(; kwargs...)
    options = Dict(
        "corr" => "squar_exp",
        "poly" => "constant",
        "theta_bounds" => [1e-6, 1e2],
        "noise0" => [1e-2],
        "hyper_opt" => "Cobyla",
        "eval_noise" => true,
        "nugget" => 1000.0 * eps(Float64),
        "method" => "FITC",
        "n_inducing" => 10
    )

    for (key, value) in kwargs
        options[key] = value
    end

    Z = get(options, "Z", rand(3, 2))  # Providing a default value for Z if not provided
    woodbury_data = get(options, "woodbury_data", Dict("vec" => rand(3), "inv" => rand(3, 3)))  # Providing default values for woodbury_data
    optimal_par = get(options, "optimal_par", Dict("theta" => rand(2), "sigma2" => rand()))  # Providing default values for optimal_par
    optimal_noise = get(options, "optimal_noise", rand())  # Providing a default value for optimal_noise

    SGP("SGP", options, Z, woodbury_data, optimal_par, optimal_noise)
end


function set_inducing_inputs!(sgp::SGP, Z::Matrix{Float64}, normalize::Bool=false)
    if size(Z, 1) != size(sgp.X, 1)
        error("Number of rows of Z must be equal to the number of rows of the input matrix")
    end

    if normalize
        # Implement normalization if needed
    end

    sgp.Z = copy(Z)
end


function _compute_K(sgp::SGP, A::Matrix{Float64}, B::Matrix{Float64}, theta::Vector{Float64}, sigma2::Float64)
    dx = A .- B  # Compute element-wise differences
    d = _componentwise_distance(sgp, dx)
    r = _correlation_types(sgp, sgp.options["corr"], theta, d)
    R = reshape(r, (size(A, 1), size(B, 1)))
    K = sigma2 * R
    return K
end



function _fitc(sgp::SGP, X::Array{Float64, 2}, Y::Array{Float64, 2}, Z::Array{Float64, 2}, theta::Array{Float64, 1}, sigma2::Float64, noise::Float64, nugget::Float64)
    Knn = fill(sigma2, size(X, 1))
    Kmm = _compute_K(sgp, Z, Z, theta, sigma2) + I * nugget
    Kmn = _compute_K(sgp, Z, X, theta, sigma2)

    U = cholesky(Kmm).L
    Ui = inv(U)
    V = Ui * Kmn

    eta2 = noise

    nu = Knn - sum(V.^2, dims=1)' + eta2
    beta = 1.0 ./ nu

    A = I + V * (beta .* V')
    L = cholesky(A).L
    Li = inv(L)

    a = Y .* beta'
    b = Li * V * a

    likelihood = -0.5 * (sum(log.(nu)) + 2.0 * sum(log.(diag(L))) + a' * Y - dot(b, b))

    LiUi = Li * Ui
    LiUiT = LiUi'
    woodbury_vec = LiUiT * b
    woodbury_inv = Ui' * Ui - LiUiT * LiUi

    return likelihood, woodbury_vec, woodbury_inv
end

function _vfe(sgp::SGP, X::Array{Float64, 2}, Y::Array{Float64, 2}, Z::Array{Float64, 2}, theta::Array{Float64, 1}, sigma2::Float64, noise::Float64, nugget::Float64)
    mean = 0
    Y .-= mean

    Kmm = _compute_K(sgp, Z, Z, theta, sigma2) + I * nugget
    Kmn = _compute_K(sgp, Z, X, theta, sigma2)

    U = cholesky(Kmm).L
    Ui = inv(U)
    V = Ui * Kmn

    beta = 1.0 ./ max.(noise, nugget)

    A = beta .* V * V'
    B = I + A
    L = cholesky(B).L
    Li = inv(L)

    b = beta * Li * V * Y

    likelihood = -0.5 * (-size(Y, 1) * log(beta) + 2.0 * sum(log.(diag(L))) + beta * dot(Y, Y) - dot(b, b) + size(Y, 1) * beta * sigma2 - sum(diag(A)))

    LiUi = Li * Ui
    Bi = I + Li' * Li
    woodbury_vec = LiUi' * b
    woodbury_inv = Ui' * Bi * Ui

    return likelihood, woodbury_vec, woodbury_inv
end

function _predict_values(sgp::SGP, x::Array{Float64, 2})
    n_features_x = size(x, 2)
    n_features_Z = size(sgp.Z, 2)
    if n_features_x != n_features_Z
        error("Number of features in x must be equal to the number of features in sgp.Z")
    end

    theta = sgp.optimal_par["theta"]
    if length(theta) != n_features_x
        error("Length of the theta vector must match the number of features in x and sgp.Z")
    end

    Kx = _compute_K(sgp, x, sgp.Z, theta, sgp.optimal_par["sigma2"])
    mu = Kx * sgp.woodbury_data["vec"]
    return mu
end


function _predict_variances(sgp::SGP, x::Array{Float64, 2})
    Kx = _compute_K(sgp, sgp.Z, x, sgp.optimal_par["theta"], sgp.optimal_par["sigma2"])
    Kxx = fill(sgp.optimal_par["sigma2"], size(x, 1))
    var = (Kxx - sum((sgp.woodbury_data["inv"]' * Kx) .* Kx, dims=1))' .+ sgp.optimal_noise
    var = max.(var, 1e-15)
    return var
end


_predict_variances (generic function with 1 method)

In [52]:
using BenchmarkTools

In [53]:
# Assign sample values for SGP
X_sgp = rand(10, 2)
Y_sgp = rand(10)
Z_sgp = rand(3, 2)
θ_sgp = rand(2)
sigma2_sgp = rand()
noise_sgp = rand()
nugget_sgp = rand()

0.3122906925667791

In [54]:
# Create an instance of SGP
sgp_instance = SGP()

SGP("SGP", Dict{String, Any}("hyper_opt" => "Cobyla", "method" => "FITC", "eval_noise" => true, "poly" => "constant", "corr" => "squar_exp", "nugget" => 2.220446049250313e-13, "noise0" => [0.01], "n_inducing" => 10, "theta_bounds" => [1.0e-6, 100.0]), [0.49939767431290083 0.08719320140650544; 0.00762973725878191 0.02605838810346406; 0.17293385677642048 0.1572087261263786], Dict{String, Any}("vec" => [0.4955302398047676, 0.5647264285719458, 0.6754476078296348], "inv" => [0.14925874908913805 0.4062406545453081 0.5102084418937607; 0.57330552680985 0.787300049471846 0.03115872433620004; 0.240734223963317 0.3788392074574267 0.1340756822828979]), Dict{String, Any}("theta" => [0.20077895109436905, 0.640775114094781], "sigma2" => 0.9540648751646907), 0.018604213213631482)

In [56]:
# Assuming X_sgp has dimensions (10, 2)
num_rows_Xsgp = size(X_sgp, 1)

# Repeat sgp_instance.Z vertically to match X_sgp
repeated_Z = vcat(sgp_instance.Z, repeat(sgp_instance.Z, num_rows_Xsgp - 1))

# Set the modified Z in sgp_instance
sgp_instance.Z = repeated_Z


ErrorException: setfield!: immutable struct of type SGP cannot be changed

In [43]:
@btime _predict_values(sgp_instance, X_sgp)

DimensionMismatch: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 10 and 3